In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import json

# Load the JSON file 
brands_json = []
with open('brands.json', 'r') as file:
    for line in file:
        brands_json.append(json.loads(line))

# Function to process JSON data into a DataFrame
def brands_json_to_df(json_data):
    # Create a list to hold processed records
    records = []
    
    for item in json_data:
        # Extract all necessary fields into a single dictionary
        record = {
            "product_id": item["_id"]["$oid"],              
            "name": item["name"],                          
            "barcode": item.get("barcode"),         
            "category": item.get("category"),       
            "category_code": item.get("categoryCode"),  
            "brand_code": item.get("brandCode"),    
            "top_brand": item.get("topBrand"),      
            "cpg_id": item["cpg"]["$id"]["$oid"],          
            "ref": item["cpg"]["$ref"]                     
        }
        records.append(record)
    
    # Create a single DataFrame
    df = pd.DataFrame(records)
    return df

# Call the function to process JSON data
brands_df = brands_json_to_df(brands_json)

In [2]:
# Load the JSON file 
users_json = []
with open('users.json', 'r') as file:
    for line in file:
        users_json.append(json.loads(line))

# Function to process JSON data into a DataFrame
def users_json_to_df(json_data):
    # Create a list to hold processed records
    user_records = []
    
    for item in json_data:
        # Extract all necessary fields, handling missing fields with .get()
        user_record = {
            "user_id": item["_id"]["$oid"],                              # Unique user ID
            "active": item["active"],                                   # Active status
            "created_date": datetime.utcfromtimestamp(item["createdDate"]["$date"] / 1000),  # Convert timestamp to datetime
            "last_login": datetime.utcfromtimestamp(item["lastLogin"]["$date"] / 1000) if "lastLogin" in item else np.nan,  # Handle optional field
            "role": item["role"],                                       # User role
            "sign_up_source": item.get("signUpSource", np.nan),         # Handle missing signUpSource
            "state": item.get("state", np.nan)                          # Handle missing state
        }
        user_records.append(user_record)
    
    # Create DataFrame
    users_df = pd.DataFrame(user_records)
    return users_df

# Call the function to process JSON data
users_df = users_json_to_df(users_json)

In [3]:
receipts_json = []
with open('receipts.json', 'r') as file:
    for line in file:
        receipts_json.append(json.loads(line))

# Step 2: Extract Receipts Table and Rewards Items Table
receipts = []
rewards_items = []

for record in receipts_json:
    # Main receipt fields
    receipt = {
        "receipt_id": record.get("_id", {}).get("$oid"),
        "bonusPointsEarned": record.get("bonusPointsEarned"),
        "bonusPointsReason": record.get("bonusPointsEarnedReason"),
        "createDate": record.get("createDate", {}).get("$date"),
        "dateScanned": record.get("dateScanned", {}).get("$date"),
        "finishedDate": record.get("finishedDate", {}).get("$date"),
        "modifyDate": record.get("modifyDate", {}).get("$date"),
        "pointsAwardedDate": record.get("pointsAwardedDate", {}).get("$date"),
        "pointsEarned": record.get("pointsEarned"),
        "purchaseDate": record.get("purchaseDate", {}).get("$date"),
        "purchasedItemCount": record.get("purchasedItemCount"),
        "rewardsReceiptStatus": record.get("rewardsReceiptStatus"),
        "totalSpent": record.get("totalSpent"),
        "userId": record.get("userId")
    }
    receipts.append(receipt)
    
    # Rewards Receipt Items Table
    for item in record.get("rewardsReceiptItemList", []):
        rewards_items.append({
            "receipt_id": record.get("_id", {}).get("$oid"),  # Foreign key
            "barcode": item.get("barcode"),
            "brandCode":item.get("brandCode"),
            "description": item.get("description"),
            "finalPrice": item.get("finalPrice"),
            "itemPrice": item.get("itemPrice"),
            "partnerItemId": item.get("partnerItemId"),
            "quantityPurchased": item.get("quantityPurchased"),
            "rewardsGroup": item.get("rewardsGroup"),
            "rewardsProductPartnerId": item.get("rewardsProductPartnerId"),
            "pointsEarned": item.get("pointsEarned"),
            "pointsPayerId": item.get("pointsPayerId"),
            "needsFetchReview": item.get("needsFetchReview"),
            "userFlaggedBarcode": item.get("userFlaggedBarcode"),
            "userFlaggedPrice": item.get("userFlaggedPrice"),
            "userFlaggedQuantity": item.get("userFlaggedQuantity"),
            "userFlaggedNewItem": item.get("userFlaggedNewItem"),
        })

# Step 3: Convert to DataFrame
receipts_df = pd.DataFrame(receipts)
rewards_items_df = pd.DataFrame(rewards_items)

Data Quality Check - Brands Table

In [5]:
# Check missing value
missing_values = brands_df.isnull().sum()
missing_percentage = brands_df.isnull().mean() * 100
print("Missing Values:\n", missing_values)
print("Missing Percentage:\n", missing_percentage)

Missing Values:
 product_id         0
name               0
barcode            0
category         155
category_code    650
brand_code       234
top_brand        612
cpg_id             0
ref                0
dtype: int64
Missing Percentage:
 product_id        0.000000
name              0.000000
barcode           0.000000
category         13.281919
category_code    55.698372
brand_code       20.051414
top_brand        52.442159
cpg_id            0.000000
ref               0.000000
dtype: float64


In [6]:
# There are lots of missing values in category, category_code, brand_code, top_brand columns. 

In [7]:
# I observed that there are strings contains "test", it might not suitable for furthere analysis 
# check for 'test' or similar placeholders in the 'name' or 'brand_code' columns:

# Check for 'test' in the 'name' column
name_test_data = brands_df[brands_df['name'].str.contains('test', case=False, na=False)]
name_test_count = len(name_test_data)
name_test_percentage = (name_test_count / len(brands_df)) * 100

# Check for 'test' in the 'brand_code' column
brand_code_test_data = brands_df[brands_df['brand_code'].str.contains('test', case=False, na=False)]
brand_code_test_count = len(brand_code_test_data)
brand_code_test_percentage = (brand_code_test_count / len(brands_df)) * 100

# Output the results
print(f"Number of rows containing 'test' in 'name': {name_test_count}")
print(f"Percentage of rows containing 'test' in 'name': {name_test_percentage:.2f}%")

print(f"Number of rows containing 'test' in 'brand_code': {brand_code_test_count}")
print(f"Percentage of rows containing 'test' in 'brand_code': {brand_code_test_percentage:.2f}%")

Number of rows containing 'test' in 'name': 432
Percentage of rows containing 'test' in 'name': 37.02%
Number of rows containing 'test' in 'brand_code': 360
Percentage of rows containing 'test' in 'brand_code': 30.85%


In [8]:
# Create a copy of the DataFrame
brand_df_copy = brands_df.copy()

# Normalize formats in the copied table
brand_df_copy['category_normalized'] = brand_df_copy['category'].str.lower().str.replace(" ", "_")
brand_df_copy['category_code_normalized'] = brand_df_copy['category_code'].str.lower()

# Compare normalized values
category_set = set(brand_df_copy['category_normalized'].dropna().unique())
category_code_set = set(brand_df_copy['category_code_normalized'].dropna().unique())

# Find mismatched values
only_in_category = category_set - category_code_set
only_in_category_code = category_code_set - category_set

# Output the results
print("Values in 'category' but not in 'category_code':", only_in_category)
print("Values in 'category_code' but not in 'category':", only_in_category_code)

Values in 'category' but not in 'category_code': {'breakfast_&_cereal', 'condiments_&_sauces', 'household', 'beauty_&_personal_care', 'canned_goods_&_soups', 'candy_&_sweets', 'health_&_wellness', 'dairy', 'dairy_&_refrigerated', 'cleaning_&_home_improvement', 'deli', 'snacks', 'bread_&_bakery', 'beauty'}
Values in 'category_code' but not in 'category': {'bread_and_bakery', 'cleaning_and_home_improvement', 'dairy_and_refrigerated', 'candy_and_sweets', 'healthy_and_wellness'}


In [9]:
# From observation we can know that mismatch exists, but seems like it is just "and" used as "&" in category secion. Those values missed 
# are actually: snacks, dairy, household, breakfast_&_cereal, canned_goods_&_soups, deli, beauty, condiments_&_sauces

Data Quality Check - Users Table

In [11]:
# Define columns to consider for identifying the same login session
key_columns = ['user_id', 'created_date', 'last_login', 'role', 'sign_up_source', 'state']

# Find exact duplicated rows based on the key columns
duplicated_rows = users_df[users_df.duplicated(subset=key_columns, keep=False)]

# Count and display duplicate rows
print(f"Number of duplicated rows: {len(duplicated_rows)}")

# Group duplicates by user_id 
grouped_duplicates = duplicated_rows.groupby('user_id').size()
print("\nNumber of duplicate rows per user:")
print(grouped_duplicates.sort_values(ascending=False))

Number of duplicated rows: 353

Number of duplicate rows per user:
user_id
54943462e4b07e684157a532    20
5fc961c3b8cfca11a077dd33    20
5ff5d15aeb7c7d12096d91a2    18
59c124bae4b0299e55b0f330    18
5fa41775898c7a11a6bcef3e    18
                            ..
5ff7268eeb7c7d12096da2a9     2
5ff73b90eb7c7d31ca8a452b     2
600f41b2bd196811e68ea219     2
601c2c05969c0b11f7d0b097     2
60229990b57b8a12187fe9e0     2
Length: 70, dtype: int64


There are duplicated login sessions based on the list of users provided, this might affect further analysis

In [13]:
# Check for missing values across df
missing_values = users_df.isnull().sum()
missing_percentage = (missing_values / len(users_df)) * 100
print("\nMissing Values (Count and Percentage):")
print(missing_values)
print(missing_percentage)


Missing Values (Count and Percentage):
user_id            0
active             0
created_date       0
last_login        62
role               0
sign_up_source    48
state             56
dtype: int64
user_id            0.000000
active             0.000000
created_date       0.000000
last_login        12.525253
role               0.000000
sign_up_source     9.696970
state             11.313131
dtype: float64


In [14]:
# Check for inconsistent values in role column
print("\nUnique values in 'role':")
print(users_df['role'].unique())


Unique values in 'role':
['consumer' 'fetch-staff']


We should only consider to use value as "consumer" in role column to analyze the data, "fetch-staff" will affect further analysis

In [16]:
# Validate that created_date is earlier than or equal to last_login
invalid_dates = users_df[users_df['created_date'] > users_df['last_login']]
print(f"\nNumber of records where created_date is after last_login: {len(invalid_dates)}")
if len(invalid_dates) > 0:
    print("Examples of invalid date records:")
    print(invalid_dates)


Number of records where created_date is after last_login: 0


Data Quality Check - Reciept table & Rewards Item Table

In [18]:
# Check for missing values
missing_values = receipts_df.isnull().sum()
missing_percentage = (missing_values / len(receipts_df)) * 100
print("Missing Values (Count and Percentage):")
print(missing_values)
print(missing_percentage)

Missing Values (Count and Percentage):
receipt_id                0
bonusPointsEarned       575
bonusPointsReason       575
createDate                0
dateScanned               0
finishedDate            551
modifyDate                0
pointsAwardedDate       582
pointsEarned            510
purchaseDate            448
purchasedItemCount      484
rewardsReceiptStatus      0
totalSpent              435
userId                    0
dtype: int64
receipt_id               0.000000
bonusPointsEarned       51.385165
bonusPointsReason       51.385165
createDate               0.000000
dateScanned              0.000000
finishedDate            49.240393
modifyDate               0.000000
pointsAwardedDate       52.010724
pointsEarned            45.576408
purchaseDate            40.035746
purchasedItemCount      43.252904
rewardsReceiptStatus     0.000000
totalSpent              38.873995
userId                   0.000000
dtype: float64


In [19]:
# Check for invalid or inconsistent dates
receipts_df['createDate'] = pd.to_datetime(receipts_df['createDate'], unit='ms', errors='coerce')
receipts_df['dateScanned'] = pd.to_datetime(receipts_df['dateScanned'], unit='ms', errors='coerce')
receipts_df['finishedDate'] = pd.to_datetime(receipts_df['finishedDate'], unit='ms', errors='coerce')

# Verify date consistency
invalid_dates = receipts_df[(receipts_df['createDate'] > receipts_df['dateScanned']) | 
                            (receipts_df['dateScanned'] > receipts_df['finishedDate'])]
print("\nNumber of rows with invalid dates:", len(invalid_dates))
print("Invalid date rows:")
print(invalid_dates)


Number of rows with invalid dates: 0
Invalid date rows:
Empty DataFrame
Columns: [receipt_id, bonusPointsEarned, bonusPointsReason, createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, pointsEarned, purchaseDate, purchasedItemCount, rewardsReceiptStatus, totalSpent, userId]
Index: []


In [20]:
# Check for duplicated receipt_ids
duplicate_receipts = receipts_df[receipts_df.duplicated(subset=['receipt_id'], keep=False)]
print("\nNumber of duplicated receipt_ids:", len(duplicate_receipts))
print("Duplicated receipts:")
print(duplicate_receipts)


Number of duplicated receipt_ids: 0
Duplicated receipts:
Empty DataFrame
Columns: [receipt_id, bonusPointsEarned, bonusPointsReason, createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, pointsEarned, purchaseDate, purchasedItemCount, rewardsReceiptStatus, totalSpent, userId]
Index: []


In [21]:
# Convert columns to numeric, coercing errors (non-numeric values will become NaN)
numeric_columns = ['bonusPointsEarned', 'pointsEarned', 'purchasedItemCount', 'totalSpent']
for col in numeric_columns:
    receipts_df[col] = pd.to_numeric(receipts_df[col], errors='coerce')

# Check for negative values after conversion
negative_values = receipts_df[(receipts_df['bonusPointsEarned'] < 0) |
                               (receipts_df['pointsEarned'] < 0) |
                               (receipts_df['purchasedItemCount'] < 0) |
                               (receipts_df['totalSpent'] < 0)]

print("\nNumber of rows with negative values in numeric columns:", len(negative_values))
print("Negative value rows:")
print(negative_values)


Number of rows with negative values in numeric columns: 0
Negative value rows:
Empty DataFrame
Columns: [receipt_id, bonusPointsEarned, bonusPointsReason, createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, pointsEarned, purchaseDate, purchasedItemCount, rewardsReceiptStatus, totalSpent, userId]
Index: []


In [22]:
# Check for missing values in each column
missing_values = rewards_items_df.isnull().sum()
missing_percentage = (missing_values / len(rewards_items_df)) * 100
print("Missing Values (Count and Percentage):")
print(missing_values)
print(missing_percentage)

Missing Values (Count and Percentage):
receipt_id                    0
barcode                    3851
brandCode                  4341
description                 381
finalPrice                  174
itemPrice                   174
partnerItemId                 0
quantityPurchased           174
rewardsGroup               5210
rewardsProductPartnerId    4672
pointsEarned               6014
pointsPayerId              5674
needsFetchReview           6128
userFlaggedBarcode         6604
userFlaggedPrice           6642
userFlaggedQuantity        6642
userFlaggedNewItem         6618
dtype: int64
receipt_id                  0.000000
barcode                    55.481919
brandCode                  62.541421
description                 5.489123
finalPrice                  2.506843
itemPrice                   2.506843
partnerItemId               0.000000
quantityPurchased           2.506843
rewardsGroup               75.061230
rewardsProductPartnerId    67.310186
pointsEarned               86.6445

In [23]:
# Ensure numeric columns are properly converted to numeric
numeric_columns = ['finalPrice', 'itemPrice', 'pointsEarned', 'userFlaggedPrice', 'userFlaggedQuantity']
for col in numeric_columns:
    rewards_items_df[col] = pd.to_numeric(rewards_items_df[col], errors='coerce')

# Identify rows with negative or invalid numeric values
invalid_numeric = rewards_items_df[(rewards_items_df[numeric_columns] < 0).any(axis=1)]
print("\nRows with negative or invalid numeric values:")
print(invalid_numeric)


Rows with negative or invalid numeric values:
Empty DataFrame
Columns: [receipt_id, barcode, brandCode, description, finalPrice, itemPrice, partnerItemId, quantityPurchased, rewardsGroup, rewardsProductPartnerId, pointsEarned, pointsPayerId, needsFetchReview, userFlaggedBarcode, userFlaggedPrice, userFlaggedQuantity, userFlaggedNewItem]
Index: []


In [24]:
# Check for duplicate rows based on all columns
duplicated_rows_all_columns = rewards_items_df[rewards_items_df.duplicated(keep=False)]

# Print the number of duplicated rows and the rows themselves
print("\nNumber of duplicated rows (all columns):", len(duplicated_rows_all_columns))
print("Duplicated rows (all columns):")
print(duplicated_rows_all_columns)


Number of duplicated rows (all columns): 0
Duplicated rows (all columns):
Empty DataFrame
Columns: [receipt_id, barcode, brandCode, description, finalPrice, itemPrice, partnerItemId, quantityPurchased, rewardsGroup, rewardsProductPartnerId, pointsEarned, pointsPayerId, needsFetchReview, userFlaggedBarcode, userFlaggedPrice, userFlaggedQuantity, userFlaggedNewItem]
Index: []


In [25]:
# Convert all numeric columns to proper numeric types
numeric_columns = rewards_items_df.select_dtypes(include=['number']).columns

# Identify rows with negative values in any numeric column
negative_values = rewards_items_df[(rewards_items_df[numeric_columns] < 0).any(axis=1)]

# Print the number of rows with negative values and the rows themselves
print("\nNumber of rows with negative values in numeric columns:", len(negative_values))
print("Rows with negative values:")
print(negative_values)


Number of rows with negative values in numeric columns: 0
Rows with negative values:
Empty DataFrame
Columns: [receipt_id, barcode, brandCode, description, finalPrice, itemPrice, partnerItemId, quantityPurchased, rewardsGroup, rewardsProductPartnerId, pointsEarned, pointsPayerId, needsFetchReview, userFlaggedBarcode, userFlaggedPrice, userFlaggedQuantity, userFlaggedNewItem]
Index: []


SQL Queries to Answer Following Business Questions:

Which brand has the most spend among users who were created within the past 6 months?
Which brand has the most transactions among users who were created within the past 6 months?

In [40]:
import pandasql as ps

To be clarified: since today is 01/19/2025, when I check the data I realized that there will not be users create accounts within six months (days after 07/22/2024) from the table provided, I will assume to use the latest login date as today, which is 2021-03-05. The SQL dialect will be MySQL

In [66]:

query = """

SELECT 
    b.name AS brand_name,
    SUM(r1.totalSpent) AS total_spend
FROM 
    receipts_df r1
LEFT JOIN 
    rewards_items_df r2 
    ON r1.receipt_id = r2.receipt_id
LEFT JOIN 
    brands_df b 
    ON r2.barcode = b.barcode AND r2.brandCode = b.brand_code
WHERE 
    r1.userId IN (
        SELECT DISTINCT 
            user_id
        FROM 
            users_df
        WHERE 
            created_date >= (
                SELECT 
                    DATETIME(MAX(last_login), '-6 months') 
                FROM 
                    users_df
                WHERE 
                    role = 'consumer'
            )
    )
    AND b.name IS NOT NULL
GROUP BY 
    b.name
ORDER BY 
    total_spend DESC
"""

# Execute the query
result = ps.sqldf(query, locals())
result

,brand_name,total_spend
0,Tostitos,15799.37
1,Pepperidge Farm,14165.85
2,V8,9443.90
3,Prego,9443.90
4,Swanson,7187.14
5,Cracker Barrel Cheese,4885.89
6,Jell-O,4721.95
7,Cheetos,4721.95
8,Kettle Brand,2400.91
9,Grey Poupon,743.79


Which brand has the most spend among users who were created within the past 6 months?
Answer: Tostitos

In [70]:
query_1 = """

SELECT 
    b.name AS brand_name,
    count(distinct r1.receipt_id) AS transaction_counts
FROM 
    receipts_df r1
LEFT JOIN 
    rewards_items_df r2 
    ON r1.receipt_id = r2.receipt_id
LEFT JOIN 
    brands_df b 
    ON r2.barcode = b.barcode AND r2.brandCode = b.brand_code
WHERE 
    r1.userId IN (
        SELECT DISTINCT 
            user_id
        FROM 
            users_df
        WHERE 
            created_date >= (
                SELECT 
                    DATETIME(MAX(last_login), '-6 months') 
                FROM 
                    users_df
                WHERE 
                    role = 'consumer'
            )
    )
    AND b.name IS NOT NULL
GROUP BY 
    b.name
ORDER BY 
    transaction_counts DESC
"""

# Execute the query
result_1 = ps.sqldf(query_1, locals())
result_1

,brand_name,transaction_counts
0,Tostitos,11
1,Swanson,11
2,Kettle Brand,3
3,Cracker Barrel Cheese,2
4,V8,1
5,Prego,1
6,Pepperidge Farm,1
7,Jell-O,1
8,Grey Poupon,1
9,Cheetos,1


Which brand has the most transactions among users who were created within the past 6 months?
Answer: Tostitos and Swanson